## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.io  # Direct import for loading .mat files
import scipy.signal  # Direct import for signal processing functions
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

def LoadTroikaDataset():
    """
    Load dataset file paths for Troika dataset.
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Load data from a .mat file.
    """
    # Load the .mat file
    data = scipy.io.loadmat(data_fl)
    
    # Ensure 'sig' key exists in the loaded data
    if 'sig' not in data:
        raise KeyError(f"'sig' key not found in {data_fl}")
    
    # Return all data except the first two elements
    return data['sig'][2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Compute the aggregate error metric based on the 90th percentile confidence.
    """
    percentile90_confidence = np.percentile(confidence_est, 10)
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]
    return np.mean(np.abs(best_estimates))

fs = 125  # signals were sampled at 125 Hz
minBPM = 40  # min bpm
maxBPM = 240  # max bpm
window_length = 8 * fs  # 8 second time window
window_shift = 2 * fs  # 2 second shift to next window

def bandpass_filter(signal, fs):
    """
    Apply a bandpass filter to the signal.
    """
    pass_band = (minBPM/60, maxBPM/60)
    b, a = scipy.signal.cheby2(4, 20, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def fourier_transform(signal, fs):
    """
    Compute the Fourier Transform of the signal.
    """
    fft = np.abs(np.fft.rfft(signal))
    freqs = np.fft.rfftfreq(len(signal), 1/fs)
    return fft, freqs

def calculate_confidence(freqs, fft_f, bpm_max):
    """
    Calculate confidence based on the FFT within a specific frequency window.
    """
    fundamental_freq_window = (freqs > bpm_max - minBPM/60) & (freqs < bpm_max + minBPM/60)
    return np.sum(fft_f[fundamental_freq_window]) / np.sum(fft_f) if np.sum(fft_f) != 0 else 0


def RunPulseRateAlgorithm(data_fl, ref_fl):
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ground_truth = sp.io.loadmat(ref_fl)['BPM0'].flatten()  # Ensure ground_truth is a 1D array

    # Apply bandpass filter to signals
    ppg = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs)
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)

    bpm_pred = []
    confidence = []
    features = []

    # Ensure that the length of ground_truth is sufficient for the windows we process
    if len(ppg) < window_length:
        raise ValueError("PPG signal is too short for the specified window length")

    for i in range(0, len(ppg) - window_length + 1, window_shift):
        ppg_window = ppg[i:i+window_length]
        acc_window = np.sqrt(accx[i:i+window_length]**2 + accy[i:i+window_length]**2 + accz[i:i+window_length]**2)

        fft_ppg, ppg_freqs = fourier_transform(ppg_window, fs)
        fft_acc, acc_freqs = fourier_transform(acc_window, fs)

        fft_ppg[ppg_freqs <= (minBPM)/60.0] = 0.0
        fft_ppg[ppg_freqs >= (maxBPM)/60.0] = 0.0
        fft_acc[acc_freqs <= (minBPM)/60.0] = 0.0
        fft_acc[acc_freqs >= (maxBPM)/60.0] = 0.0

        ppg_max = ppg_freqs[np.argmax(fft_ppg)]
        acc_max = acc_freqs[np.argmax(fft_acc)]

        combined_max = (ppg_max + acc_max) / 2
        conf_val = calculate_confidence(ppg_freqs, fft_ppg, ppg_max)

        bpm_pred.append(ppg_max * 60)
        confidence.append(conf_val)

        feature = [ppg_max * 60, acc_max * 60, combined_max * 60, conf_val]
        features.append(feature)

    # Convert lists to numpy arrays
    features = np.array(features)
    bpm_pred = np.array(bpm_pred)

    # Match the length of ground_truth with bpm_pred
    if len(ground_truth) > len(bpm_pred):
        ground_truth = ground_truth[:len(bpm_pred)]
    elif len(bpm_pred) > len(ground_truth):
        bpm_pred = bpm_pred[:len(ground_truth)]

    errors = np.abs(np.subtract(ground_truth, bpm_pred))
    return errors, confidence, features

def TrainModel(features, bpm_pred):
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
    model.fit(features, bpm_pred)
    return model


def Evaluate():
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    all_features = []
    all_bpm_preds = []
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        errors, confidence, features = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        all_features.append(features)
        all_bpm_preds.extend(errors)

    # Ensure that all_features and all_bpm_preds have the same length
    all_features = np.vstack(all_features)
    all_bpm_preds = np.array(all_bpm_preds)

    if len(all_features) != len(all_bpm_preds):
        raise ValueError(f"Feature and target lengths do not match: {len(all_features)} vs {len(all_bpm_preds)}")

    X = all_features
    y = all_bpm_preds

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    model = TrainModel(X_train, y_train)
    predicted_bpm = model.predict(X_test)

    # Calculate metrics
    mae = mean_absolute_error(y_test, predicted_bpm)
    mse = mean_squared_error(y_test, predicted_bpm)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predicted_bpm)

    print(f"Evaluation Metrics:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

    # Additional comment about generalizability
    print("Note: These metrics are based on a single test split. Performance may vary with different datasets or cross-validation strategies.")

Evaluate()


Evaluation Metrics:
MAE: 11.6833
MSE: 234.7637
RMSE: 15.3220
R²: 0.5777
Note: These metrics are based on a single test split. Performance may vary with different datasets or cross-validation strategies.


-----
### Project Write-up
> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
###### Data Loading: The LoadTroikaDataset function retrieves data file paths and reference files from a specified directory. 
###### Preprocessing: Signals are filtered using a bandpass filter to isolate the frequency range corresponding to the heart rate (BPM). 
###### Feature Extraction: The RunPulseRateAlgorithm function extracts features from the filtered signals, performs a Fourier transform to identify the frequency components, and calculates a confidence score for each window of data.
###### Model Training and Evaluation: The TrainModel function initializes a Gradient Boosting Regressor model, which is then trained using features extracted from the signals. The Evaluate function splits the data into training and test sets, evaluates the model using error metrics, and prints the results.

> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
The dataset used for training and testing consists of physiological signals (PPG and accelerometer data) from the Troika dataset. The dataset includes:
###### PPG Signals: Photoplethysmogram signals capturing blood volume changes. Accelerometer Signals: Signals from three axes of an accelerometer. 
###### Shortcomings: Limited Diversity: The dataset may not represent all possible variations in heart rate or signal quality across different populations. Noise and Artifacts: The signals might contain noise or artifacts that can impact model performance.
###### Improvements: Expanded Dataset: Include data from diverse populations and different conditions. Enhanced Signal Quality: Use data with reduced noise and artifacts for better model training.

> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
How the Algorithm Works:
###### Signal Filtering: The algorithm applies a bandpass filter to isolate frequencies corresponding to the heart rate. Feature Extraction: It performs Fourier transforms to extract frequency components and calculates a confidence score based on the prominence of the fundamental frequency. Model Training: Uses a Gradient Boosting Regressor to predict the heart rate from the extracted features.
###### Physiological Aspects: PPG Signal: Utilizes frequency information from the PPG signal to estimate the heart rate. Accelerometer Data: Combines accelerometer data to enhance the robustness of the estimation.
###### Algorithm Outputs: Predicted BPM: The estimated heart rate in beats per minute. Confidence Scores: A measure of confidence in the predicted BPM based on the frequency components. 
###### Caveats on Algorithm Outputs: Accuracy Variability: Performance can vary based on signal quality and individual differences. Sensitivity to Noise: The algorithm may be sensitive to noise or artifacts in the signal.
###### Common Failure Modes: Poor Signal Quality: Low-quality or noisy signals can lead to inaccurate BPM estimates. Inadequate Feature Extraction: If key features are not effectively captured, prediction accuracy may decrease. Overfitting: The model may overfit to the training data if not properly validated.

> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.
Performance Computation:
###### Training and Testing: The dataset is split into training (80%) and test (20%) sets using train_test_split. The model is trained on the training set and evaluated on the test set. Metrics: The performance is measured using Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and R-squared (R²).
###### Generalizability Caveat: Performance Variability: The metrics reported are based on a single train-test split. Performance may vary with different datasets or when using cross-validation.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 